# **Bioinformatics Project - Computational Drug Discovery [Part 6] Making Predictions**

Juan Oliveira

Na **Parte 6**, realizaremos a predição e teste do modelo



In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
! unzip padel.zip
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2024-07-01 23:37:22--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2024-07-01 23:37:22--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M   162MB/s    in 0.2s    

2024-07-01 23:37:22 (162 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2024-07-01 23:37:22--  https://github

# Carregar dados de bioatividade

In [ ]:
!pip install pandas --upgrade # Atualiza a biblioteca pandas para a versão mais recente
import pandas as pd

Requirement already up-to-date: pandas in /usr/local/lib/python3.7/site-packages (1.3.5)


Bibliotecas

In [ ]:
import numpy as np
!pip install rdKit
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

proj_path = '/content/gdrive/MyDrive/Colab Notebooks/'

file_name2='molecules_for_inference.csv'
df = pd.read_csv(proj_path + file_name2)
df

     |████████████████████████████████| 29.5 MB 2.1 MB/s 
Mounted at /content/gdrive


,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive


Calculando os Descritores

In [ ]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

In [ ]:
df_lipinski = lipinski(df.canonical_smiles)

Vamos dar uma olhada nos 2 DataFrames que serão combinados.

In [ ]:
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,467.61,4.3603,1.0,5.0
1,467.61,4.3603,1.0,5.0
2,467.61,4.3603,1.0,5.0
3,467.61,4.3603,1.0,5.0


In [ ]:
df

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive


Agora, vamos combinar os 2 DataFrame

In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)
df_combined

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive,467.61,4.3603,1.0,5.0
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive,467.61,4.3603,1.0,5.0
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive,467.61,4.3603,1.0,5.0
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,10000.0,inactive,467.61,4.3603,1.0,5.0


Converter IC50 para pIC50

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', axis=1)

    return x


In [ ]:
df_combined.standard_value.describe()

count        4.0
mean     10000.0
std          0.0
min      10000.0
25%      10000.0
50%      10000.0
75%      10000.0
max      10000.0
Name: standard_value, dtype: float64

In [ ]:
-np.log10( (10**-9)* 100000000 )

1.0

In [ ]:
-np.log10( (10**-9)* 10000000000 )

-1.0

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', axis=1)

    return x

In [ ]:
df_norm = norm_value(df_combined)
df_norm

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,10000.0
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,10000.0
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,10000.0
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,10000.0


In [ ]:
df_norm.standard_value_norm.describe()

count        4.0
mean     10000.0
std          0.0
min      10000.0
25%      10000.0
50%      10000.0
75%      10000.0
max      10000.0
Name: standard_value_norm, dtype: float64

In [ ]:
df_final = pIC50(df_norm)
df_final

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0


In [ ]:
df_final.pIC50.describe()

count    4.0
mean     5.0
std      0.0
min      5.0
25%      5.0
50%      5.0
75%      5.0
max      5.0
Name: pIC50, dtype: float64

In [ ]:
file_name3='VHC_08_bioactivity_data_for_prediction.csv'
df_final.to_csv(proj_path + file_name3)  # Salva o DataFrame 'df_final' em um arquivo CSV com o nome 'VHC_08_bioactivity_data_for_prediction.csv

Carregar dado de bioatividade

In [ ]:
file_name4='VHC_08_bioactivity_data_for_prediction.csv'
df3 = pd.read_csv(proj_path + file_name4)
df3

,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0
1,1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0
2,2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0
3,3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,5.0


In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule_inference.smi', sep='\t', index=False, header=False) # Salva os dados selecionados em um arquivo SMI (formato de arquivo químico) chamado 'molecule_inference.smi'

In [ ]:
! cat molecule.smi | head -5 # exibir as primeiras cinco linhas

cat: molecule.smi: No such file or directory


In [ ]:
! cat molecule.smi | wc -l # contando quantas linhas tem nesse arquivo

cat: molecule.smi: No such file or directory
0


Calcular Descritores de fingerprint

In [ ]:
! cat padel.sh # comando para executar o script
! bash padel.sh # executar o script
! ls -l # verificar se o arquivo foi criado

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv
Processing CHEMBL113494 in molecule_inference.smi (1/4). 
Processing CHEMBL113494 in molecule_inference.smi (2/4). 
Processing CHEMBL113494 in molecule_inference.smi (3/4). 
Processing CHEMBL113494 in molecule_inference.smi (4/4). Average speed: 2.55 s/mol.
Descriptor calculation completed in 6.793 secs . Average speed: 1.70 s/mol.
total 108276
-rw-r--r-- 1 root root    18456 Jul  1 23:40 descriptors_output.csv
drwx------ 6 root root     4096 Jul  1 23:39 gdrive
drwxr-xr-x 3 root root     4096 Jul  1 23:39 __MACOSX
-rwxr-xr-x 1 root root 85055499 Mar 11  2020 Miniconda3-py37_4.8.2-Linux-x86_64.sh
-rw-r--r-- 1 root root      328 Jul  1 23:39 molecule_inference.smi
drwxrwxr-x 4 root root     4096 May 30  2020 PaDEL-Descriptor
-rw-r--r-- 1 root root      231 Jul

Preparando as Matrizes de Dados X e Y

X data matrix

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv')  # leitura do dados de um arquivo CSV
df3_X # impressão da tabela

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,CHEMBL113494,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL113494,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL113494,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL113494,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df3_X = df3_X.drop(columns=['Name'])## tiramos a coluna 'name', pois precisamos deixar somente os dados quantitativos
df3_X


,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Variavel Y

Convert IC50 to pIC50

In [ ]:
# Cria uma nova série chamada 'df3_Y' com os valores da coluna 'pIC50'
df3_Y = df3['pIC50']
df3_Y

0    5.0
1    5.0
2    5.0
3    5.0
Name: pIC50, dtype: float64

Combining X and Y variable

In [ ]:
dataset3 = pd.concat([df3_X,df3_Y], axis=1) # Concatena as duas tabelas em uma única tabela
dataset3 # Imprime a tabela resultante

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.0


In [ ]:
file_name5='VHC_09_bioactivity_data_3class_pIC50_pubchem_fp.csv' # salvando o dataframe 'dataset3' em um arquivo CSV
dataset3.to_csv(proj_path + file_name5, index=False)

juntando as duas tabelas df3

In [ ]:
df3 = df3.drop(columns=['pIC50', 'Unnamed: 0']) # Remove as colunas 'pIC50' e 'Unnamed: 0' do DataFrame 'df3'
dffinal = pd.concat([df3,df3_X,df3_Y], axis=1) # Concatena as duas tabelas em uma única tabela
dffinal

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,PubchemFP0,PubchemFP1,PubchemFP2,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0


Salvando em arquivo csv

In [ ]:
file_name6='VHC_10_bioactivity_data_3class_pIC50_pubchem_fp.csv' # salvando o dataframe 'dffinal' em um arquivo CSV
dffinal.to_csv(proj_path + file_name6, index=False) # Salva o DataFrame 'dffinal' em um arquivo CSV com o nome 'VHC_10_bioactivity_data_3class_pIC50_pubchem_fp.csv'

lendo dataset

In [ ]:
df1 = dffinal
df1

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,PubchemFP0,PubchemFP1,PubchemFP2,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0
1,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0
2,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0
3,CHEMBL113494,CCCCN(CCCC)C(=O)CN1C[C@H](c2cccnc2)[C@@H](C(=O...,inactive,467.61,4.3603,1.0,5.0,1,1,1,...,0,0,0,0,0,0,0,0,0,5.0


Aplicando melhor modelo nos nossos dados: regressao

In [ ]:
# Remover colunas 'canonical_smiles', 'molecule_chembl_id e bioactivity_class'
df1 = df1.drop(columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class'])

# Remover linhas com valores nulos
df1 = df1.dropna()

# Exibir o DataFrame resultante
df1

,MW,LogP,NumHDonors,NumHAcceptors,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,5.0
1,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,5.0
2,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,5.0
3,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,5.0


dividindo Dfs em X e Y

In [ ]:
Y_our_data = df1.filter(['pIC50'], axis=1) # Cria uma nova série chamada 'Y_our_data' contendo os valores da coluna 'pIC50'
X_our_data = df1.drop('pIC50', axis=1) # Cria uma nova tabela chamada 'X_our_data' contendo todas as colunas exceto a coluna 'pIC50'
X_our_data # Imprime a tabela 'X_our_data'

,MW,LogP,NumHDonors,NumHAcceptors,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,467.61,4.3603,1.0,5.0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pickle
filename2 = 'finalized_model1.sav'
loaded_model = pickle.load(open(proj_path + filename2, 'rb'))

file_name3='VHC_selected_features.txt'
selected_columns = np.loadtxt(proj_path + file_name3, dtype=str, delimiter=',')
selected_columns = [column.strip() for column in selected_columns]

XXX = pd.DataFrame (X_our_data, columns = selected_columns)
X_our_data=XXX

#Realizando a predição
y_pred=loaded_model.predict(X_our_data)
df3 = pd.DataFrame(y_pred, columns = ['Prediction'])

df3.to_excel((proj_path + "02_Predicoes_Dados_Proprios.xlsx"), sheet_name='Predicoes')

df3

,Prediction
0,5.280981
1,5.280981
2,5.280981
3,5.280981
